# Master Thesis First Version of the STT Model 

**Author**: Karin Thommen

**Date**: March 2023 


---

**Content of the Notebook**: Train and test STT model (Spontaneous and prepared Speech) 

---

In [ ]:
# references: 
# https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLS_R_on_Common_Voice.ipynb#scrollTo=72737oog2F6U 
# 
# 

## Import and Setup

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr 12 04:31:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
%%capture
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install torch
!pip install datasets
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer
!pip install audio-metadata

In [2]:
pip install "dill<0.3.5"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.4 which is incompatible.


In [ ]:
#!pip install git+https://github.com/huggingface/transformers

In [5]:
import pandas as pd
import os
import transformers
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

from datasets.fingerprint import Hasher
import pickle
import dill

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json

import IPython.display as ipd
import numpy as np
import random

import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import audio_metadata

from datasets import load_dataset, Audio, load_metric
from datasets import Dataset

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from huggingface_hub import notebook_login

In [3]:
# Build connection to data folder on GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load train, test and validation files 
#train=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/train.tsv",sep='\t')
#test=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/test.tsv",sep='\t')
#valid=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/valid.tsv",sep='\t')

In [6]:
# login to huggingface account for data
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
# load dataset from huggingface (after uploading it via local machine to huggingface)
dataset = load_dataset("karinthommen/sds200")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3636 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3638 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/135271 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/karinthommen___parquet/karinthommen--sds200-a1893d366d27240a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Train Model on Prepared Speech

## Prepare Data

In [8]:
# check if data loading worked
dataset["train"][0]

{'audio': {'path': '09966c7743291ccf1129c8136143bf5a6132947fe352795bc6d5456a3afeb4de.mp3',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.58690691e-05, -6.36559753e-06, -1.80013558e-05]),
  'sampling_rate': 32000},
 'transcription': 'Dadurch wird auch der Lebensraum von vielen Tier- und Pflanzenarten zerstört.',
 'canton': None,
 'duration': 6.732}

In [9]:
# filter data since model could have some problems with data with more than 5sec duration 
dataset["train"] = dataset["train"].filter(lambda example: example["duration"] <= 5)

Filter:   0%|          | 0/135271 [00:00<?, ? examples/s]

In [10]:
# remove columns from dataset that we do not need at the moment 
dataset = dataset.remove_columns(["canton", "duration"])

In [11]:
# reference of some code snippets: https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLS_R_on_Common_Voice.ipynb#scrollTo=72737oog2F6U 

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

### Remove Special Characters and clean all datasets

In [12]:
# Remove special characters 

chars = '[\'̈\’\•\‹\₂\›\–\²\½\‑\°\`\&\(\)\*\+\/\=\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_chars(batch):
  batch["sentence"] = re.sub(chars, '', batch["transcription"]).lower()
  return batch

# do for whole datasets 
dataset = dataset.map(remove_special_chars)

Map:   0%|          | 0/3636 [00:00<?, ? examples/s]

Map:   0%|          | 0/3638 [00:00<?, ? examples/s]

Map:   0%|          | 0/84771 [00:00<?, ? examples/s]

In [13]:
show_random_elements(dataset["train"].remove_columns(["audio"]))

,transcription,sentence
0,Dieser Bericht ist äusserst kritisch.,dieser bericht ist äusserst kritisch
1,Darin möchten wir sie ermutigen.,darin möchten wir sie ermutigen
2,Dann bräuchten die SBB weniger Steuergelder.,dann bräuchten die sbb weniger steuergelder
3,VW hat in den USA einen guten Ruf.,vw hat in den usa einen guten ruf
4,Daran dürfte sich nicht allzu schnell etwas ändern.,daran dürfte sich nicht allzu schnell etwas ändern
5,Weil er hat alles gegeben für Allah.,weil er hat alles gegeben für allah
6,Allerdings ist der 1. August stark von Mythen geprägt.,allerdings ist der 1 august stark von mythen geprägt
7,Darauf kehrte er in die Wohnung des Paares zurück.,darauf kehrte er in die wohnung des paares zurück
8,Ausnahmen gibt es nur für die Swiss.,ausnahmen gibt es nur für die swiss
9,Die Freude hielt jedoch nicht lange.,die freude hielt jedoch nicht lange


### Get Vocabulary

In [14]:
dataset["train"][0]

{'audio': {'path': '09e565e54189cecbfbdb1085e0b2c52d4db417de42b9f665122a48765b257900.mp3',
  'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.53588935e-05, 1.78251066e-05, 6.87433931e-06]),
  'sampling_rate': 32000},
 'transcription': 'Seit dem Umbau ist das anders.',
 'sentence': 'seit dem umbau ist das anders'}

In [15]:
def extract_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

# extract the characters from all datasets 
vocab = dataset.map(extract_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=['audio', 'transcription', 'sentence'])

Map:   0%|          | 0/3636 [00:00<?, ? examples/s]

Map:   0%|          | 0/3638 [00:00<?, ? examples/s]

Map:   0%|          | 0/84771 [00:00<?, ? examples/s]

In [16]:
vocab

DatasetDict({
    test: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    train: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
})

In [17]:
# make a vocabulary list out of all three data sets and enumerate all characters and save them enumerated in a dictionary 
vocab_list = list(set(vocab["train"]["vocab"][0]) | set(vocab["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 'a': 11,
 'b': 12,
 'c': 13,
 'd': 14,
 'e': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'i': 19,
 'j': 20,
 'k': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'q': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'x': 34,
 'y': 35,
 'z': 36,
 '\xad': 37,
 'à': 38,
 'á': 39,
 'ä': 40,
 'è': 41,
 'é': 42,
 'ë': 43,
 'í': 44,
 'ö': 45,
 'ü': 46}

In [18]:
# this cell only once after the above cell. Do not rerun this cell without rerunning the above cell. 

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "] # replace the empty string in the vocabulary with a character that is better visible 
vocab_dict["[UNK]"] = len(vocab_dict) # add unknown token 
vocab_dict["[PAD]"] = len(vocab_dict) # add padding token or blank token (important for CTC algorithm)
print("Length of the vocabulary:", len(vocab_dict))

Length of the vocabulary: 49


In [19]:
# save vocabulary file for later usage 
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

### Load tokenizer, feature extractor and processor

In [22]:
# load processor and model from pretrained Wav2Vec model from huggingface
#processor = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-xls-r-300m")
#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-xls-r-300m")

In [20]:
# load vocabulary with Wav2Vec Tokenizer 
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
# instantiate feature extractor from Wav2Vec2 Feature Extractor with sampling rate of 16kHz. 
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [22]:
# after loading the tokenizer and the feature extractor, both get wrapped in a processor 
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Prepare audio clips

In [23]:
dataset["train"][0]["audio"]

{'path': '09e565e54189cecbfbdb1085e0b2c52d4db417de42b9f665122a48765b257900.mp3',
 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.53588935e-05, 1.78251066e-05, 6.87433931e-06]),
 'sampling_rate': 32000}

In [24]:
# resample audio to 16kHz. 
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

In [25]:
rand_int = random.randint(0, len(dataset["train"])-1)

print(dataset["train"][rand_int]["sentence"])
ipd.Audio(data=dataset["train"][rand_int]["audio"]["array"], autoplay=True, rate=16000)

nach der störung von gestern sieht das wetter heute ruhiger aus


In [26]:
rand_int = random.randint(0, len(dataset["train"])-1)

print("Target text:", dataset["train"][rand_int]["sentence"])
print("Input array shape:", dataset["train"][rand_int]["audio"]["array"].shape)
print("Sampling rate:", dataset["train"][rand_int]["audio"]["sampling_rate"])

Target text: aber das allein bin ich nicht
Input array shape: (61056,)
Sampling rate: 16000


In [27]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [28]:
dataset = dataset.map(prepare_dataset, remove_columns=['audio', 'transcription', 'sentence'])

Map:   0%|          | 0/3636 [00:00<?, ? examples/s]

Map:   0%|          | 0/3638 [00:00<?, ? examples/s]

Map:   0%|          | 0/84771 [00:00<?, ? examples/s]

In [32]:
dataset["train"][0]

{'input_values': [-3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-05,
  -3.962791743106209e-

In [ ]:
#dataset.push_to_hub("karinthommen/sds200-features-incl-vocab", private=True)

### Training

In [29]:
 @dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2ForCTC
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
wer_metric = load_metric("wer")

    load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate


In [32]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = 100* wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [33]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_hid.weight', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [34]:
model.freeze_feature_extractor()

In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/Masterarbeit/XLSR-V2",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=100,
  save_total_limit=2,
  push_to_hub=False,
)

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend


In [37]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 84771
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 26490
    Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
    __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_

Step,Training Loss,Validation Loss,Wer
400,3.519800,1.874232,97.604053
800,1.435500,1.237370,77.025648
1200,1.080300,1.082823,67.790170
1600,0.959800,0.972377,60.289202


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 3638
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/Masterarbeit/XLSR-V2/checkpoint-400
Configuration saved in /content/drive/MyDrive/Masterarbeit/XLSR-V2/checkpoint-400/config.json
Model weights saved in /content/drive/MyDrive/Masterarbeit/XLSR-V2/checkpoint-400/pytorch_model.bin
Configuration saved in /content/drive/MyDrive/Masterarbeit/XLSR-V2/checkpoint-400/preprocessor_config.json
Deleting older checkpoint [/content/drive/MyDrive/Masterarbeit/XLSR-V2/checkpoint-100] due to args.save_total_limit
Deleting older checkpoint [/content/drive/MyDrive/Masterarbeit/XLSR-V2/checkpoint-200] due to args.save_total_limit
    __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). Th

KeyboardInterrupt: ignored

## Test Model on Prepared Speech
